In [48]:
!pip install pymongo
!pip install pandas
import pymongo
NEW_MONGO_HOST = 'mongodb+srv://cluster0.glnjpi9.mongodb.net/test'

def get_db_handle(db_name, host, port, username, password):

    client = pymongo.MongoClient(host=host,
                         port=int(port),
                         username=username,
                         password=password)
    db_handle = client['db_name']
    return db_handle, client

def get_db_default_handle():
    db_handle, client = get_db_handle(db_name='fashion_finder_db',
                                      host=NEW_MONGO_HOST,
                                      username='django_db_user',
                                      password='Ko4mNy6A5JEaST', # TODO: make secret
                                      port=27017)
    return db_handle, client

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 1.2 MB/s eta 0:00:0000:0100:01
  Using cached numpy-1.23.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)


In [4]:
db_handle, client = get_db_default_handle()

In [5]:
db = client.fashion_finder_db
recs_coll = db.LabeledFashionPiece


In [79]:
BANNED_TONES = [
[60, 46, 40],
[75, 57, 50],
[90, 69, 60],
[105, 80, 70],
[120, 92, 80],
[135, 103, 90],
[150, 114, 100],
[165, 126, 110],
[180, 138, 120],
[195, 149, 130],
[210, 161, 140],
[225, 172, 150],
[240, 184, 160],
[255, 195, 170],
[255, 206, 180],
[255, 255, 255],

]

In [80]:
import numpy as np
def calculate_min_distance_from_skin_tone(rgb, skin_tones=BANNED_TONES):
	def calculate_euclidean_distance(rgb1, rgb2):
		rgb_0_diff = rgb1[0] - rgb2[0]
		rgb_1_diff = rgb1[1] - rgb2[1]
		rgb_2_diff = rgb1[2] - rgb2[2]
		return np.sqrt(rgb_0_diff ** 2 + rgb_1_diff ** 2 + rgb_2_diff ** 2)
	min_diff = 255**3
	for skin_tone in skin_tones:
		diff = calculate_euclidean_distance(rgb, skin_tone)
		if(diff < min_diff):
			min_diff = diff
	return min_diff


In [81]:
rgbs = [[104, 88, 87], [40, 32, 32], [223, 194, 179], [98, 93, 90], [255, 255, 254], [252,253,251], [33, 107, 235], [20, 30, 40]]

for rgb in rgbs:
	min_diff = calculate_min_distance_from_skin_tone(rgb)
	print(rgb)
	print(1 / min_diff)


[104, 88, 87]
0.05581455721859476
[40, 32, 32]
0.03892494720807615
[223, 194, 179]
0.03651483716701107
[98, 93, 90]
0.041344911529736156
[255, 255, 254]
1.0
[252, 253, 251]
0.18569533817705186
[33, 107, 235]
0.0056392870361735925
[20, 30, 40]
0.023211917272131483


In [57]:
n = 10
import itertools
erp = {'rgb_0': [104, 88, 87], 'rgb_1': [40, 32, 32], 'rgb_2': [223, 194, 179]}
rgb_0 = erp['rgb_0']
rgb_1 = erp['rgb_1']
rgb_2 = erp['rgb_2']




rgb_0_diffs = np.sum([abs(e[1] - e[0]) for e in itertools.combinations(rgb_0, 2)])
rgb_1_diffs = np.sum([abs(e[1] - e[0]) for e in itertools.combinations(rgb_1, 2)])
rgb_2_diffs = np.sum([abs(e[1] - e[0]) for e in itertools.combinations(rgb_2, 2)])
print(rgb_0_diffs, rgb_1_diffs, rgb_2_diffs)
# for arr in erp.values():

	# avg_diff = np.mean(diffs)
	# print(avg_diff)
# erp_rgb_0_orig_score = np.sum([erp])
agg_results = recs_coll.aggregate([
	{
		"$project": {
			"_id": 1,
			# RGB_SOURCE_TARGET_RGB_INDEX_diff
			# input_idx = 0, candidate_idx = 0
			# "rgb_self_diff": {"$subtract": [ {"$toInt":{"$arrayElemAt":["$rgb_0", 0]}}, {"$toInt":{"$arrayElemAt":["$rgb_0", 1]}} ] },
			# "rgb_self_diff": {"$abs": { "$subtract":[ { "$toDouble": "$rgb_0.0" }, { "$toDouble": "$rgb_0.1" } ] } },

			# "rgb_self_diff": {"$subtract": [{"$arrayElemAt": ["$rgb_0", 0]}, erp["rgb_0"][0] ] },
			# 0 0
			"rgb_0_0_0_diff": {"$subtract": [{"$arrayElemAt": ["$rgb_0", 0]}, erp["rgb_0"][0] ] },
			"rgb_0_0_1_diff": {"$subtract": [{"$arrayElemAt": ["$rgb_0", 1]}, erp["rgb_0"][1] ] },
			"rgb_0_0_2_diff": {"$subtract": [{"$arrayElemAt": ["$rgb_0", 2]}, erp["rgb_0"][2] ] },
			
			# 0 1
			"rgb_0_1_0_diff": {"$subtract": [{"$arrayElemAt": ["$rgb_0", 0]}, erp["rgb_1"][0] ] },
			"rgb_0_1_1_diff": {"$subtract": [{"$arrayElemAt": ["$rgb_0", 1]}, erp["rgb_1"][1] ] },
			"rgb_0_1_2_diff": {"$subtract": [{"$arrayElemAt": ["$rgb_0", 2]}, erp["rgb_1"][2] ] },

			# 0 2
			"rgb_0_2_0_diff": {"$subtract": [{"$arrayElemAt": ["$rgb_0", 0]}, erp["rgb_2"][0] ] },
			"rgb_0_2_1_diff": {"$subtract": [{"$arrayElemAt": ["$rgb_0", 1]}, erp["rgb_2"][1] ] },
			"rgb_0_2_2_diff": {"$subtract": [{"$arrayElemAt": ["$rgb_0", 2]}, erp["rgb_2"][2] ] },

			# 1 0
			"rgb_1_0_0_diff": {"$subtract": [{"$arrayElemAt": ["$rgb_1", 0]}, erp["rgb_0"][0] ] },
			"rgb_1_0_1_diff": {"$subtract": [{"$arrayElemAt": ["$rgb_1", 1]}, erp["rgb_0"][1] ] },
			"rgb_1_0_2_diff": {"$subtract": [{"$arrayElemAt": ["$rgb_1", 2]}, erp["rgb_0"][2] ] },

			# 1 1
			"rgb_1_1_0_diff": {"$subtract": [{"$arrayElemAt": ["$rgb_1", 0]}, erp["rgb_1"][0] ] },
			"rgb_1_1_1_diff": {"$subtract": [{"$arrayElemAt": ["$rgb_1", 1]}, erp["rgb_1"][1] ] },
			"rgb_1_1_2_diff": {"$subtract": [{"$arrayElemAt": ["$rgb_1", 2]}, erp["rgb_1"][2] ] },

			# 1 2
			"rgb_1_2_0_diff": {"$subtract": [{"$arrayElemAt": ["$rgb_1", 0]}, erp["rgb_2"][0] ] },
			"rgb_1_2_1_diff": {"$subtract": [{"$arrayElemAt": ["$rgb_1", 1]}, erp["rgb_2"][1] ] },
			"rgb_1_2_2_diff": {"$subtract": [{"$arrayElemAt": ["$rgb_1", 2]}, erp["rgb_2"][2] ] },
			
			# 2 0
			"rgb_2_0_0_diff": {"$subtract": [{"$arrayElemAt": ["$rgb_2", 0]}, erp["rgb_0"][0] ] },
			"rgb_2_0_1_diff": {"$subtract": [{"$arrayElemAt": ["$rgb_2", 1]}, erp["rgb_0"][1] ] },
			"rgb_2_0_2_diff": {"$subtract": [{"$arrayElemAt": ["$rgb_2", 2]}, erp["rgb_0"][2] ] },

			# 2 1
			"rgb_2_1_0_diff": {"$subtract": [{"$arrayElemAt": ["$rgb_2", 0]}, erp["rgb_1"][0] ] },
			"rgb_2_1_1_diff": {"$subtract": [{"$arrayElemAt": ["$rgb_2", 1]}, erp["rgb_1"][1] ] },
			"rgb_2_1_2_diff": {"$subtract": [{"$arrayElemAt": ["$rgb_2", 2]}, erp["rgb_1"][2] ] },

			# 2 2
			"rgb_2_2_0_diff": {"$subtract": [{"$arrayElemAt": ["$rgb_2", 0]}, erp["rgb_2"][0] ] },
			"rgb_2_2_1_diff": {"$subtract": [{"$arrayElemAt": ["$rgb_2", 1]}, erp["rgb_2"][1] ] },
			"rgb_2_2_2_diff": {"$subtract": [{"$arrayElemAt": ["$rgb_2", 2]}, erp["rgb_2"][2] ] },


		},
	},
	{
		"$project": {
			"_id": 1,
			# Calculate the euclidean distance for each pairing of hex codes 
			"rgb_0_0_euclid": {"$sum": [
				{"$multiply": ["$rgb_0_0_0_diff", "$rgb_0_0_0_diff"]},
				{"$multiply": ["$rgb_0_0_1_diff", "$rgb_0_0_1_diff"]},
				{"$multiply": ["$rgb_0_0_2_diff", "$rgb_0_0_2_diff"]},
			]},

			"rgb_0_1_euclid": {"$sum": [
				{"$multiply": ["$rgb_0_1_0_diff", "$rgb_0_1_0_diff"]},
				{"$multiply": ["$rgb_0_1_1_diff", "$rgb_0_1_1_diff"]},
				{"$multiply": ["$rgb_0_1_2_diff", "$rgb_0_1_2_diff"]},
			]},

			"rgb_0_2_euclid": {"$sum": [
				{"$multiply": ["$rgb_0_2_0_diff", "$rgb_0_2_0_diff"]},
				{"$multiply": ["$rgb_0_2_1_diff", "$rgb_0_2_1_diff"]},
				{"$multiply": ["$rgb_0_2_2_diff", "$rgb_0_2_2_diff"]},
			]},

			"rgb_1_0_euclid": {"$sum": [
				{"$multiply": ["$rgb_1_0_0_diff", "$rgb_1_0_0_diff"]},
				{"$multiply": ["$rgb_1_0_1_diff", "$rgb_1_0_1_diff"]},
				{"$multiply": ["$rgb_1_0_2_diff", "$rgb_1_0_2_diff"]},
			]},

			"rgb_1_1_euclid": {"$sum": [
				{"$multiply": ["$rgb_1_1_0_diff", "$rgb_1_1_0_diff"]},
				{"$multiply": ["$rgb_1_1_1_diff", "$rgb_1_1_1_diff"]},
				{"$multiply": ["$rgb_1_1_2_diff", "$rgb_1_1_2_diff"]},
			]},

			"rgb_1_2_euclid": {"$sum": [
				{"$multiply": ["$rgb_1_1_0_diff", "$rgb_1_1_0_diff"]},
				{"$multiply": ["$rgb_1_1_1_diff", "$rgb_1_1_1_diff"]},
				{"$multiply": ["$rgb_1_1_2_diff", "$rgb_1_1_2_diff"]},
			]},

			"rgb_2_0_euclid": {"$sum": [
				{"$multiply": ["$rgb_2_0_0_diff", "$rgb_2_0_0_diff"]},
				{"$multiply": ["$rgb_2_0_1_diff", "$rgb_2_0_1_diff"]},
				{"$multiply": ["$rgb_2_0_2_diff", "$rgb_2_0_2_diff"]},
			]},

			"rgb_2_1_euclid": {"$sum": [
				{"$multiply": ["$rgb_2_1_0_diff", "$rgb_2_1_0_diff"]},
				{"$multiply": ["$rgb_2_1_1_diff", "$rgb_2_1_1_diff"]},
				{"$multiply": ["$rgb_2_1_2_diff", "$rgb_2_1_2_diff"]},
			]},

			"rgb_2_2_euclid": {"$sum": [
				{"$multiply": ["$rgb_2_2_0_diff", "$rgb_2_2_0_diff"]},
				{"$multiply": ["$rgb_2_2_1_diff", "$rgb_2_2_1_diff"]},
				{"$multiply": ["$rgb_2_2_2_diff", "$rgb_2_2_2_diff"]},
			]},


		}
	},
	{
		"$project": {
			"rgb_0_0_euclid": 1,
			"rgb_0_1_euclid": 1,
			"rgb_0_2_euclid": 1,
			"rgb_1_0_euclid": 1,
			"rgb_1_1_euclid": 1,
			"rgb_1_2_euclid": 1,
			"rgb_2_0_euclid": 1,
			"rgb_2_1_euclid": 1,
			"rgb_2_2_euclid": 1,

			# We want to reward colors which have some variation between the 3 pixel values 

			"min_0_distance": {"$min":["$rgb_0_0_euclid", "$rgb_1_0_euclid", "$rgb_2_0_euclid"]},
			"min_1_distance": {"$min":["$rgb_0_1_euclid", "$rgb_1_1_euclid", "$rgb_2_1_euclid"]},
			"min_2_distance": {"$min":["$rgb_0_2_euclid", "$rgb_1_2_euclid", "$rgb_2_2_euclid"]},
			# "min_distance": {"$min":["$rgb_0_0_euclid","$rgb_0_1_euclid","$rgb_0_2_euclid","$rgb_1_0_euclid","$rgb_1_1_euclid","$rgb_1_2_euclid","$rgb_2_0_euclid","$rgb_2_1_euclid","$rgb_2_2_euclid"]}
		}
	},
	{
		"$project": {

		}
	},

	{
		"$sort": {"min_distance": 1},
	},

	{
		"$limit": n

	}
])

34 16 88


In [55]:
for rec in agg_results:
	print(rec)

{'_id': ObjectId('635b376e82641a0935e88bd8'), 'rgb_0_0_euclid': 0, 'rgb_0_1_euclid': 10257, 'rgb_0_2_euclid': 33861, 'rgb_1_0_euclid': 10257, 'rgb_1_1_euclid': 0, 'rgb_1_2_euclid': 0, 'rgb_2_0_euclid': 33861, 'rgb_2_1_euclid': 81342, 'rgb_2_2_euclid': 0, 'min_distance': 0}
{'_id': ObjectId('635b379e82641a0935e8d775'), 'rgb_0_0_euclid': 203, 'rgb_0_1_euclid': 10290, 'rgb_0_2_euclid': 34158, 'rgb_1_0_euclid': 10368, 'rgb_1_1_euclid': 1, 'rgb_1_2_euclid': 1, 'rgb_2_0_euclid': 62629, 'rgb_2_1_euclid': 123246, 'rgb_2_2_euclid': 5328, 'min_distance': 1}
{'_id': ObjectId('635b377382641a0935e8946d'), 'rgb_0_0_euclid': 257, 'rgb_0_1_euclid': 8354, 'rgb_0_2_euclid': 38138, 'rgb_1_0_euclid': 20705, 'rgb_1_1_euclid': 1826, 'rgb_1_2_euclid': 1826, 'rgb_2_0_euclid': 10148, 'rgb_2_1_euclid': 1, 'rgb_2_2_euclid': 81049, 'min_distance': 1}
{'_id': ObjectId('635b379882641a0935e8cdb2'), 'rgb_0_0_euclid': 57300, 'rgb_0_1_euclid': 115681, 'rgb_0_2_euclid': 4153, 'rgb_1_0_euclid': 10370, 'rgb_1_1_euclid': 1

In [39]:
rec_f = recs_coll.find({"_id":"635b376e82641a0935e88a85"})